### Import Library

In [245]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import random
from scipy.sparse import csr_matrix, vstack
# from textblob import TextBlob
# from langdetect import detect_langs
import pickle

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Import the clean data

In [246]:
song_df = pd.read_pickle('../../data_lyrics/pd/clean_th_songs_35.pkl')

In [247]:
# song_df = song_df.astype({"artist": 'category', "song_name": 'category', "href": 'string', "lyric": 'string', 'lines': 'int64', 'words': object, 'n_words': 'int64', 'words_str': 'string', 'artists': object, 'duplicates': 'int64', 'n_artists': 'int64'})
# song_df.dtypes

In [248]:
song_df.head()

,artist,song_name,href,lyric,lines,words,n_words,words_str,artists,duplicates,n_artists
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,57,"[ไม่, ว่า, จะ, เป็น, ยังไง, babi, it, okay, ไม...",343,ไม่ ว่า จะ เป็น ยังไง babi it okay ไม่ ว่า จะ ...,{bird_thongchai},0,1
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,31,"[จะ, ยอม, นั่ง, จรวด, ไป, ตรวจ, ดาวอังคาร, หาก...",239,จะ ยอม นั่ง จรวด ไป ตรวจ ดาวอังคาร หาก ว่าที่ ...,{bird_thongchai},0,1
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,39,"[ถ้า, เคย, พบ, เจอ, กำแพง, ที่, ดู, ทั้ง, ใหญ่...",316,ถ้า เคย พบ เจอ กำแพง ที่ ดู ทั้ง ใหญ่ และ สูงช...,{bird_thongchai},0,1
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,50,"[อยู่, ตรงนี้, แค่, เพียงลำพัง, กับ, ความเหงา,...",250,อยู่ ตรงนี้ แค่ เพียงลำพัง กับ ความเหงา ที่ เป...,{bird_thongchai},0,1
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,65,"[แค่, ตัว, คนเดียว, ไม่, ตาย, ล่ะ, มั้ง, ถาม, ...",416,แค่ ตัว คนเดียว ไม่ ตาย ล่ะ มั้ง ถาม ใจ กี่ คร...,{bird_thongchai},0,1


## Feature engineering

### Number of words

In [249]:
n_artist = len(song_df['artist'].unique())
random.seed(0)

artist_select = random.choices(song_df['artist'].unique(), k=n_artist)

song_filter_df = song_df.loc[song_df['artist'].isin(artist_select)]
print('Total number of songs: {}'.format(len(song_filter_df)))
song_filter_df.groupby('artist')[['song_name']].count().reset_index().rename(columns={'song_name':'n_songs'})

Total number of songs: 1295


,artist,n_songs
0,25_hours,35
1,add_carabao,35
2,ann_thitima,35
3,big_ass,35
4,bird_thongchai,35
5,bnk48,35
6,bodyslam,35
7,carabao,35
8,cocktail,35
9,da_endorphine,35


In [250]:
fig = px.box(song_filter_df, x='artist', y='n_words', title='Word count per song by artist')
fig.show()

In [251]:
song_filter_df

,artist,song_name,href,lyric,lines,words,n_words,words_str,artists,duplicates,n_artists
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,57,"[ไม่, ว่า, จะ, เป็น, ยังไง, babi, it, okay, ไม...",343,ไม่ ว่า จะ เป็น ยังไง babi it okay ไม่ ว่า จะ ...,{bird_thongchai},0,1
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,31,"[จะ, ยอม, นั่ง, จรวด, ไป, ตรวจ, ดาวอังคาร, หาก...",239,จะ ยอม นั่ง จรวด ไป ตรวจ ดาวอังคาร หาก ว่าที่ ...,{bird_thongchai},0,1
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,39,"[ถ้า, เคย, พบ, เจอ, กำแพง, ที่, ดู, ทั้ง, ใหญ่...",316,ถ้า เคย พบ เจอ กำแพง ที่ ดู ทั้ง ใหญ่ และ สูงช...,{bird_thongchai},0,1
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,50,"[อยู่, ตรงนี้, แค่, เพียงลำพัง, กับ, ความเหงา,...",250,อยู่ ตรงนี้ แค่ เพียงลำพัง กับ ความเหงา ที่ เป...,{bird_thongchai},0,1
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,65,"[แค่, ตัว, คนเดียว, ไม่, ตาย, ล่ะ, มั้ง, ถาม, ...",416,แค่ ตัว คนเดียว ไม่ ตาย ล่ะ มั้ง ถาม ใจ กี่ คร...,{bird_thongchai},0,1
...,...,...,...,...,...,...,...,...,...,...,...
2200,sek_loso,อย่ายอมแพ้ (โดยที่ยังไม่ต่อสู้),/music/thailyric/5703,เส้นทางนั้นฉันเคยเดินผ่านมา\rเส้นทางโน้นฉันก็เ...,31,"[เส้นทาง, นั้น, ฉัน, เคย, เดินผ่าน, มา, เส้น, ...",166,เส้นทาง นั้น ฉัน เคย เดินผ่าน มา เส้น ทางโน้น ...,{sek_loso},0,1
2201,sek_loso,ก้อนเนื้อข้างซ้าย,/music/thailyric/5684,ฉันเหงาเหลือเกิน\rตั้งแต่วันที่เธอจากฉันไป\rเห...,28,"[ฉัน, เหงา, เหลือเกิน, ตั้งแต่, วันที่, เธอ, จ...",170,ฉัน เหงา เหลือเกิน ตั้งแต่ วันที่ เธอ จาก ฉัน ...,{sek_loso},0,1
2202,sek_loso,ขาร็อคขาเลาะ,/music/thailyric/13264,สาวเอ้ยบ้านน้องอยู่ไส\rมันเป็นจั๋งได๋ คือมาน่า...,42,"[สาว, เอ้ย, บ้าน, น้อง, อยู่, ไส, มัน, เป็น, จ...",271,สาว เอ้ย บ้าน น้อง อยู่ ไส มัน เป็น จั๋ง ได๋ ค...,{sek_loso},0,1
2203,sek_loso,ใจสลายที่ราชประสงค์,/music/thailyric/10711,เหมือนฟ้าถล่ม ลงกลางหัวใจ\rเมื่อเธอจากไป ไม่มี...,37,"[เหมือน, ฟ้า, ถล่ม, ลง, กลาง, หัวใจ, เมื่อ, เธ...",223,เหมือน ฟ้า ถล่ม ลง กลาง หัวใจ เมื่อ เธอ จากไป ...,{sek_loso},0,1


### Repeated

In [252]:
# number of unique stems
song_df['n_unique_words'] = song_df['words'].map(lambda lst: len(set(lst)))

# ratio of unique stems
song_df['unique_words_ratio'] = song_df['n_unique_words'] / song_df['n_words']

# attach column to selected artists
song_filter_df = song_filter_df.join(song_df['unique_words_ratio'])

In [253]:
fig = px.box(song_filter_df, x='artist', y='unique_words_ratio', title='Ratio of unique words to all words')
fig.show()

### Words per line

In [254]:
# calculate number of words per line
song_df['words_per_line'] = song_df['n_words'] / song_df['lines'].astype(float)

song_filter_df = song_filter_df.join(song_df['words_per_line'])

In [255]:
fig = px.box(song_filter_df, x='artist', y='words_per_line', title='Words per line')
fig.show()

## TFIDF

In [256]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [257]:
# initialise count vectorizer
cv = CountVectorizer(analyzer=lambda x:x.split())

word_count_vector = cv.fit_transform(song_df['words_str'])

# compute idf
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [258]:
# print idf values
tfidf_df = pd.DataFrame({'word': cv.get_feature_names(), 'weight': tfidf_transformer.idf_})
 
# get lowest weights
tfidf_df.sort_values('weight').head()

c:\Users\FACT-PC\anaconda3\envs\torch38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,word,weight
14885,ไม่,1.100365
6965,ที่,1.171934
5263,จะ,1.187422
14855,ไป,1.192277
14734,ให้,1.210286


In [259]:
cv.get_feature_names()

c:\Users\FACT-PC\anaconda3\envs\torch38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



['[',
 'a',
 'about',
 'aboutask',
 'aboutthat',
 'abov',
 'abovefre',
 'abovefrozen',
 'abovehigh',
 'aboveriseris',
 'aboveso',
 'aboveth',
 'abovethey',
 'aboveyour',
 'absolut',
 'accent',
 'accenteast',
 'accessori',
 'ace',
 'achievedrap',
 'across',
 'act',
 'actionsguess',
 'ad',
 'adapt',
 'addi',
 'addict',
 'address',
 'adventur',
 'aey',
 'afraid',
 'afraidy',
 'after',
 'again',
 'againbecaus',
 'againbut',
 'againgood',
 'againlov',
 'agent',
 'agreether',
 'ah',
 'aha',
 'ahand',
 'ahead',
 'ahgiv',
 'ahh',
 'ahheat',
 'ahsh',
 'ahw',
 'ai',
 'aim',
 'aint',
 'air',
 'airlin',
 'airto',
 'aitakatta',
 'aitakattaaitakatta',
 'aixd',
 'ak',
 'album',
 'albumnow',
 'alcohol',
 'alert',
 'alexanderruth',
 'aliveheaven',
 'all',
 'alla',
 'alley',
 'alli',
 'allmi',
 'allowin',
 'allth',
 'allyou',
 'alon',
 'aloneand',
 'alonebabi',
 'alonei',
 'alonejust',
 'alonenow',
 'alonewhen',
 'along',
 'alongand',
 'alreadi',
 'alreadyso',
 'alright',
 'alrightar',
 'alrighteverybod

In [260]:
# get highest weights
tfidf_df.sort_values('weight', ascending=False).head(5)

,word,weight
0,[,8.02153
8665,ฟันฟาง,8.02153
13113,เย็นชื่น,8.02153
8904,มาดาม,8.02153
8905,มาตรการ,8.02153


In [261]:
# assign tf idf scores to each song
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

# attach count vectors to dataframe
tf_idf_vector_lst = [-1] * len(song_df)
for i in range(len(song_df)):
    tf_idf_vector_lst[i] = tf_idf_vector[i]
song_df['tf_idf_vector'] = tf_idf_vector_lst    

song_df['tf_idf_score'] = song_df['tf_idf_vector'].map(lambda vec: np.sum(vec.todense()))

# join valus to selected artists
song_filter_df = song_filter_df.join(song_df[['tf_idf_vector', 'tf_idf_score']])

In [262]:
tf_idf_vector.shape

(2240, 15042)

In [263]:
fig = px.box(song_filter_df, x='artist', y='tf_idf_score', title='TFIDF scores of songs per artist')
fig.show()

In [264]:
# calculate mean vector
def get_mean_vector(vec_lst):
    return csr_matrix(vstack(vec_lst).mean(axis=0))

In [265]:
# calculate mean vector over all songs of same artist
artist_df = song_df.groupby('artist').agg({'tf_idf_vector': get_mean_vector, 'song_name': len}).reset_index()\
                   .rename(columns={'song_name': 'n_songs'})

# get selected artists
artist_filter_df = artist_df.loc[artist_df['artist'].isin(song_filter_df['artist'])]

### Similarity of Songs

In [266]:
similarity_matrix = cosine_similarity(vstack(artist_filter_df['tf_idf_vector']), 
                                      vstack(artist_filter_df['tf_idf_vector']))
artist_names = artist_filter_df['artist'].tolist()
fig = go.Figure(data=go.Heatmap(z=np.flipud(similarity_matrix), x=artist_names, y=list(reversed(artist_names)), 
                                colorscale='balance', zmin=0.5, zmax=1.1))

fig.update_layout(
    width=1500,
    height=1000,)
    
fig.show()

In [267]:
artist_filter_df

,artist,tf_idf_vector,n_songs
0,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35
1,add_carabao,"(0, 1)\t0.01722099493446398\n (0, 32)\t0.00...",35
2,ann_thitima,"(0, 1)\t0.001205217775690307\n (0, 207)\t0....",35
6,big_ass,"(0, 592)\t0.012547968164130173\n (0, 2146)\...",35
7,bird_thongchai,"(0, 65)\t0.002947692762340396\n (0, 93)\t0....",35
8,bnk48,"(0, 40)\t0.011194993584758377\n (0, 55)\t0....",35
9,bodyslam,"(0, 3664)\t0.002646773914079035\n (0, 3717)...",35
10,carabao,"(0, 3661)\t0.0026894281128822055\n (0, 3663...",35
11,cocktail,"(0, 211)\t0.0029959778928084577\n (0, 601)\...",35
12,da_endorphine,"(0, 1)\t0.00047849201835692043\n (0, 107)\t...",35


In [268]:
song_filter_df.sample(5)

,artist,song_name,href,lyric,lines,words,n_words,words_str,artists,duplicates,n_artists,unique_words_ratio,words_per_line,tf_idf_vector,tf_idf_score
540,scrubb,หนี,/music/thailyric/18133,ในความทรงจำ ฉันยืนอยู่ในวัน\rเมื่อวานตอนที่เรา...,26,"[ใน, ความทรงจำ, ฉัน, ยืน, อยู่, ใน, วัน, เมื่อ...",151,ใน ความทรงจำ ฉัน ยืน อยู่ ใน วัน เมื่อวาน ตอนท...,{scrubb},0,1,0.397351,5.807692,"(0, 14958)\t0.10753700672422206\n (0, 14855...",6.870686
1693,25_hours,แรงโน้มถ่วง,/music/thailyric/10479,บินไปในท้องนภา ให้ไกลจนลับสายตา\rเดินทางไปบนดว...,41,"[บิน, ไป, ใน, ท้อง, นภา, ให้, ไกล, จน, ลับสายต...",310,บิน ไป ใน ท้อง นภา ให้ ไกล จน ลับสายตา เดินทาง...,{25_hours},0,1,0.229032,7.560976,"(0, 14986)\t0.060684729336324794\n (0, 1495...",2.979150
1129,petch_saharat,โง่เอง,/music/thailyric/14295,บ่แม่นดีเอสไอ เลยสืบบ่ได้ว่าคนร้ายคือเธอ\rลอบส...,37,"[บ่, แม่น, ดีเอสไอ, เลย, สืบ, บ่, ได้, ว่า, คน...",257,บ่ แม่น ดีเอสไอ เลย สืบ บ่ ได้ ว่า คนร้าย คือ ...,{petch_saharat},0,1,0.342412,6.945946,"(0, 15006)\t0.011420958483154792\n (0, 1493...",5.053104
2153,nut_meria,เปลี่ยนใจไม่ได้แล้ว,/music/thailyric/2323,เธอพูดใช่มั้ย ใช่มั้ย\rใช่มั้ย นี่เธอนึกยังไงก...,81,"[เธอ, พูด, ใช่, มั้ย, ใช่, มั้ย, ใช่, มั้ย, นี...",384,เธอ พูด ใช่ มั้ย ใช่ มั้ย ใช่ มั้ย นี่ เธอ นึก...,{nut_meria},0,1,0.247396,4.740741,"(0, 15006)\t0.01164013275703079\n (0, 14986...",6.273115
1513,da_endorphine,รักครั้งใหม่,/music/thailyric/1604,รักวันวานในความเจ็บช้ำ\rเคยทำให้คนอย่างฉันลำบา...,26,"[รัก, วันวาน, ใน, ความเจ็บช้ำ, เคย, ทำให้, คน,...",168,รัก วันวาน ใน ความเจ็บช้ำ เคย ทำให้ คน อย่าง ฉ...,{da_endorphine},0,1,0.476190,6.461538,"(0, 14973)\t0.03789787550661723\n (0, 14958...",7.577964


In [269]:
artist_song_filter_df = pd.merge(artist_filter_df[['artist', 'tf_idf_vector', 'n_songs']].assign(key = 0), 
                                 song_filter_df[['artist', 'tf_idf_vector', 'song_name']].assign(key = 0), on='key', 
                                 suffixes=['_artist', '_song']).drop('key', axis=1).reset_index(drop=True)

artist_song_filter_df['same_artist'] = artist_song_filter_df['artist_artist'] == artist_song_filter_df['artist_song']

In [270]:
artist_song_filter_df[artist_song_filter_df['same_artist']==False].head()

,artist_artist,tf_idf_vector_artist,n_songs,artist_song,tf_idf_vector_song,song_name,same_artist
0,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35,bird_thongchai,"(0, 14986)\t0.02706318124303119\n (0, 14978...",Okay,False
1,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35,bird_thongchai,"(0, 14973)\t0.07331503970888636\n (0, 14914...",กว่าจักรวาล,False
2,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35,bird_thongchai,"(0, 14978)\t0.0895602203600452\n (0, 14973)...",กำแพง,False
3,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35,bird_thongchai,"(0, 14978)\t0.08974985827768892\n (0, 14973...",ชีวิตเดี่ยว,False
4,25_hours,"(0, 107)\t0.0033729404635357454\n (0, 1071)...",35,bird_thongchai,"(0, 14973)\t0.033155545848114705\n (0, 1488...",ผู้ต้องหา,False


In [271]:
# calculate similarity of artist tf idf vector and song vector
def tf_idf_vector_similarity(artist_vector, song_vector, songs, same_artist):
    # check if song is from same artist
    if same_artist:
        # deduct song vector from artist vector
        artist_vector = (songs * artist_vector - song_vector) / (songs - 1)
    # calculate similarity
    return cosine_similarity(artist_vector, song_vector)[0][0]

In [272]:
artist_song_filter_df['vector_similarity'] = artist_song_filter_df.apply(lambda row: tf_idf_vector_similarity(row['tf_idf_vector_artist'], 
                                                                     row['tf_idf_vector_song'], 
                                                                     row['n_songs'], row['same_artist']), axis=1)

# This function may take longer than 8 minutes.

In [273]:
df = artist_song_filter_df

fig = go.Figure()

fig.add_trace(go.Violin(x=df['artist_artist'][df['same_artist']],
                        y=df['vector_similarity'][df['same_artist']],
                        legendgroup='Same Artist', scalegroup='Same Artist', name='Same Artist',
                        side='negative')
             )
fig.add_trace(go.Violin(x=df['artist_artist'][~df['same_artist']],
                        y=df['vector_similarity'][~df['same_artist']],
                        legendgroup='Different Artists', scalegroup='Different Artists', name='Different Artists',
                        side='positive')
             )


fig.update_layout(
    width=2000,
    height=800,)
fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay')
fig.update_layout(title='Similarity of Songs')
fig.update_xaxes(range=[-0.5, 9.5])
fig.update_yaxes(range=[-0.1, 0.8], title='Similarity')
fig.show()

# Note that you should click *Autoscale* on the figure option to show all artists' violins

## Sentiment analysis

In [274]:
# polarity_lst = [-1] * len(song_df)
# subjectivity_lst = [-1] * len(song_df)

# for i, text in enumerate(song_df['lyric']):
#     sentiment = TextBlob(text)
#     polarity_lst[i] = sentiment.polarity
#     subjectivity_lst[i] = sentiment.subjectivity
    
# song_df['polarity'] = polarity_lst
# song_df['subjectivity'] = subjectivity_lst

# song_filter_df = song_filter_df.join(song_df[['polarity', 'subjectivity']])

### Polarity and Subjectivity of Songs

In [275]:
# fig = px.scatter(song_filter_df, x='polarity', y='subjectivity', color='artist', hover_data=['song_name'], title='Polarity and Subjectivity of Songs')
# fig.show()

### Polarity by artist

In [276]:
# fig = px.box(song_filter_df, x='artist', y='polarity', title='Polarity by artist')
# fig.show()

In [277]:
# song_filter_df

## Export the feature-extracted data

In [278]:
song_df.to_pickle('../../data_lyrics/pd/fx_th_songs_35.pkl')